In [8]:
from collections import defaultdict
import datetime
from gensim import corpora, models, similarities
from matplotlib import pyplot as plt
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
from sklearn.preprocessing import MultiLabelBinarizer

from tqdm import tqdm_notebook

import src.knntools as knntools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [10]:
training_email_ids_per_sender = preprocess.get_email_ids_per_sender(training)
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)

In [11]:
training_body_dict = preprocess.body_dict_from_panda(training_info)
test_body_dict = preprocess.body_dict_from_panda(test_info)


In [12]:
training_token_dict = texttools.get_token_dict(training_body_dict)
test_token_dict = texttools.get_token_dict(test_body_dict)

In [13]:
training_email_list, training_idx_to_mids = texttools.token_dicts_to_token_lists(training_token_dict, rarity_threshold=3)
test_email_list, test_idx_to_mids = texttools.token_dicts_to_token_lists(test_token_dict, rarity_threshold=3)

In [14]:
# Assign unique id to word
training_word_id_dic = corpora.Dictionary(training_email_list)

# Compute email corpush as bow [[(wordid_1_1, count_1_1), ...] ...]
training_email_corpus = [training_word_id_dic.doc2bow(text) for text in training_email_list]
test_email_corpus = [training_word_id_dic.doc2bow(text) for text in test_email_list]

In [15]:
nb_topics = 200
nb_training_emails = len(training_email_corpus)
print(nb_training_emails)

43613


In [16]:
# model_vars_path = 'variables/hdp_similarities_{thres_nb}_words_out_no_punct'.format(thres_nb=rarity_thres)
stored_model_results = ['variables/training-lda-200.p']

lda_model = texttools.compute_model(training_email_corpus, training_word_id_dic, model='lda', nb_topics=nb_topics,
                                    use_saved=True, save=True, model_results_path=stored_model_results[0])

this will take some time...
computed model
saving new model to variables/training-lda-200.p


In [33]:
training_lda_vectors = lda_model[training_email_corpus]
test_lda_vectors = lda_model[test_email_corpus]

In [54]:
len(test_lda_vectors)

2181

In [57]:
training_lda_feature_matrix = texttools.model_vectors_to_matrix_features(training_lda_vectors, nb_topics)

In [58]:
test_lda_feature_matrix = texttools.model_vectors_to_matrix_features(test_lda_vectors, nb_topics)

In [64]:
lda_model.print_topics(2,10)

[(58,
  '0.069*"edison" + 0.065*"sce" + 0.056*"bankruptcy" + 0.028*"petition" + 0.025*"renewable" + 0.024*"qf" + 0.022*"eix" + 0.019*"pcg" + 0.019*"creditors" + 0.016*"filing"'),
 (89,
  '0.098*"jim" + 0.068*"karen" + 0.046*"presentation" + 0.040*"steve" + 0.035*"fyi" + 0.034*"denne" + 0.033*"--" + 0.033*"page" + 0.020*"derrick" + 0.017*"palmer"')]

In [107]:
training_day_features = recency.get_sparse_date_info(training_idx_to_mids, training_info)
test_day_features = recency.get_sparse_date_info(test_idx_to_mids, test_info)

In [108]:
print(test_day_features.shape)
print(test_day_features.shape)

(41432, 7)
(2181, 7)


In [114]:
training_sender_idx_dic = preprocess.get_sender_idx_dics(training_email_ids_per_sender)
training_sender_features = preprocess.get_sparse_sender_info(training_idx_to_mids, training_sender_idx_dic,
                                       training_email_ids_per_sender, training_info)
test_sender_features = preprocess.get_sparse_sender_info(test_idx_to_mids, training_sender_idx_dic,
                                       test_email_ids_per_sender, test_info)

In [113]:
print(train_sender_features.shape)
print(val_sender_features.shape)
print(np.sum(train_sender_features))
print(np.sum(val_sender_features))


(41432, 125)
(2181, 125)
41432.0
2181.0


In [121]:
stacked_training_features = scipy.sparse.hstack([training_sender_features, training_day_features, training_lda_feature_matrix])

In [122]:
stacked_train_features.shape

(41432, 332)

In [ ]:
stacked_test_features = scipy.sparse.hstack([test_sender_features, test_day_features, test_lda_feature_matrix])

In [18]:
current_score = scoring.compute_prediction_mad(hdp_predictions, val_info)
print(current_score)

0.0807795545579
